In [1]:
import os
import numpy as np
import pandas as pd

In [20]:
path_to_file = "../videos/lemur_ids_full/batch_cleaned1_22/A_e2_c3"
#path_to_file = "../videos/lemur_id_test/cleaned1/all"
experiment = "A_e2_c3"

In [7]:
data_dict = {}

# Open the text document and read it line by line
with open(os.path.join(path_to_file, experiment +'.txt'), 'r') as file:
    for line_number, line in enumerate(file):
        elements = line.strip().split(',') 
        data_dict[line_number] = elements

# Create a DataFrame from the dictionary
df = pd.DataFrame.from_dict(data_dict, orient='index')

df.columns = ["frame", "track_id", "V3", "V4", "V5", "V6", "conf", "class", "Cha", "Flo", "Gen", "Geo", "Her", "Rab", "Red", "Uns", "ID"]
df.head()


,frame,track_id,V3,V4,V5,V6,conf,class,Cha,Flo,Gen,Geo,Her,Rab,Red,Uns,ID
0,523,1,-48.198890686035156,74.70600891113281,799.4013061523438,865.0142211914062,0.6633123755455017,0,0.012809801,0.0041741855,0.30891186,0.2784166,0.029449489,0.009642742,0.121922836,0.23467249,2
1,524,1,-61.218327279573145,-15.736597959660344,738.6449809146312,799.3722839355469,0.02459883876144886,0,0.009946237,0.0051829983,0.051138896,0.36954328,0.0538675,0.0033024382,0.027194459,0.47982424,7
2,525,1,-96.86144467232936,-36.18435885017277,682.8192352710279,742.582743498301,0.18942776322364807,0,0.008318401,0.004065953,0.00863822,0.18233645,0.023433013,0.00251976,0.024499409,0.74618876,7
3,526,1,-78.37516875101232,-81.76917696807959,565.6842126024495,618.3884322335582,0.059372831135988235,0,0.008809717,0.011812563,0.03152954,0.02965427,0.04776055,0.026796013,0.03462746,0.8090098,7
4,798,2,1730.1927490234375,-58.638160705566406,117.39794921874999,223.93795776367188,0.6600873470306396,0,0.024547797,0.024702424,0.043622993,0.10462457,0.061198257,0.059293285,0.059869602,0.62214106,7


In [8]:
def process_dataframe(df):
    # Select columns from "frame" to "class"
    selected_columns = df.columns[:df.columns.get_loc("class") + 1]

    # Concatenate selected columns with "ID" column
    new_df = pd.concat([df[selected_columns], df["ID"]], axis=1)

    # Calculate maximum among columns "Cha" to "Uns"
    max_value = df.loc[:, "Cha":"Uns"].max(axis=1)

    # Add the calculated maximum as a new column to the DataFrame
    new_df["conf_id"] = max_value

    int_columns = ['frame', 'track_id', 'class']
    new_df[int_columns] = new_df[int_columns].astype(int)

    


    # Convert specific columns to double/float
    double_columns = ['V3', 'V4', 'V5', 'V6', 'conf', 'conf_id']
    new_df[double_columns] = new_df[double_columns].astype(float)
    new_df = new_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    return new_df


In [9]:
new_df = process_dataframe(df)
new_df.sample(10)

,frame,track_id,V3,V4,V5,V6,conf,class,ID,conf_id
3423,1391,1,622.815842,295.827744,440.617186,396.059580,0.948861,1,None,NaN
3019,1026,1,623.919432,297.362116,438.533306,393.613916,0.956026,1,None,NaN
1,524,1,-61.218327,-15.736598,738.644981,799.372284,0.024599,0,7,0.479824
2177,603,2,-45.253710,277.394394,269.180201,306.124826,0.793823,1,None,NaN
2674,852,1,609.884462,295.634753,450.002765,394.769317,0.927817,1,None,NaN
2398,714,1,610.242544,295.157669,449.627019,395.567795,0.926440,1,None,NaN
529,1297,7,1717.098659,1.740822,114.903745,161.000179,0.863405,0,7,0.806952
64,861,2,1484.665206,436.864981,253.446703,221.322850,0.900871,0,5,0.865449
3683,1617,1,651.521070,301.701478,322.348567,393.229491,0.796814,1,None,NaN
662,1341,6,147.866900,246.279613,396.189159,352.854561,0.966296,0,4,0.965514


In [10]:
def calculate_weight(confidence):
    return np.exp(9.2 * (confidence - 0.5))

In [14]:
def majority_vote(group):
    weighted_ids = {}
    counted_ids = {}
    total_weight = 0
    total_length = 0
    for index, row in group.iterrows():
        weight = calculate_weight(row['conf_id'])
        total_weight += weight
        total_length += 1
        if row['ID'] != '7':  # Exclude ID 7 from majority vote
            if row['ID'] in weighted_ids:
                weighted_ids[row['ID']] += weight
                counted_ids[row['ID']] += 1 
            else:
                weighted_ids[row['ID']] = weight
                counted_ids[row['ID']] = 1 
            
    if len(weighted_ids) > 0:
        max_weight_id = max(weighted_ids, key=weighted_ids.get)
        max_weight_count = counted_ids[str(max_weight_id)]
        max_weight_score = weighted_ids[max_weight_id]
    
    if (len(weighted_ids) ==0) or (max_weight_score < 1) or (total_length < 10):
        max_weight_id = '7'
        max_weight_count = 0
        max_weight_score = 0
    
    return max_weight_id, max_weight_count, max_weight_score, total_length

In [15]:
result = new_df[new_df['class'] == 0].groupby('track_id').apply(majority_vote)

results_df = result.apply(pd.Series)

# Set the index name
results_df.index.name = 'track_id'

# Rename the columns
results_df.columns = ['new_ID', 'count', 'weighted_score', 'track_length']

print(results_df[:30])

         new_ID  count  weighted_score  track_length
track_id                                            
1             7      0        0.000000             4
2             5     86     2274.939663           136
3             4    122     2219.781814           133
4             6    245     4576.875853           660
5             7      0        0.000000             2
6             4    142     6606.547912           174
7             2     90     1020.594689           525
8             7      0        0.000000            11


In [16]:
replacement_dict = results_df['new_ID'].to_dict()

In [17]:
output_df = new_df.copy()

output_df.loc[(output_df['class'] == 0) & (output_df['track_id'].isin(replacement_dict.keys())), 'ID'] = \
    output_df.loc[(output_df['class'] == 0) & (output_df['track_id'].isin(replacement_dict.keys())), 'track_id'].map(replacement_dict)

In [18]:
output_df.sample(100)
output_df.to_csv(os.path.join(path_to_file, experiment + "_post.txt"), header=False, index=False)

### Visualize

In [22]:
def get_color(id):
        if id == 0:
            color = (25, 25, 180)
        elif id == 1:
            color = (50, 0, 50)
        elif id == 2:
            color = (125, 30, 30)
        elif id == 3:
            color = (0, 175, 50)
        elif id == 4:
            color = (100, 100, 180)
        elif id == 5:
            color = (200, 0, 50)
        elif id == 6:
            color = (20, 180, 20)
        else:
            color = (0, 0, 0)
        return color

In [23]:
import cv2
import numpy as np


indivs = ['Cha', 'Flo', 'Gen', 'Geo', 'Her', 'Rab', 'Red', 'Uns']

# Load the video
video_path = '/path/to/video/A_e2_c3.mp4'
cap = cv2.VideoCapture(video_path)

# Define the output video writer
output_path = '../videos/postprocessing/' + experiment + '_post.mp4'
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Read the data from the text file
data_file = os.path.join(path_to_file, experiment + "_post.txt")
with open(data_file, 'r') as f:
    lines = f.readlines()

# Create a dictionary to store bounding boxes for each frame
bounding_boxes = {}
for line in lines:
    values = line.strip().split(',')
    frame_number = int(values[0])
    bbox_left = float(values[2])
    bbox_top= float(values[3])
    bbox_width = float(values[4])
    bbox_height = float(values[5])
    bbox_conf = float(values[6])
    class_id = int(values[7])
    indiv_id = values[8]
    #ID_conf = float(values[9])

    # Add bounding box to dictionary
    if frame_number not in bounding_boxes:
        bounding_boxes[frame_number] = []
    bounding_boxes[frame_number].append((bbox_left, bbox_top, bbox_width, bbox_height, class_id, indiv_id))

# Iterate through each frame in the video
for frame_number in range(1, int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) + 1):
    # Read the frame from the video
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()

    # Draw bounding boxes for the current frame
    if frame_number in bounding_boxes:
        for bbox_left, bbox_top, bbox_width, bbox_height, class_id, indiv_id in bounding_boxes[frame_number]:
            top_left = (int(bbox_left), int(bbox_top))
            bottom_right = (int(bbox_left + bbox_width), int(bbox_top + bbox_height))
            if class_id == 0:
                color = get_color(int(indiv_id))  # Color dependent on ID
            else:
                color = (255, 255, 255)
            thickness = 3
            cv2.rectangle(frame, top_left, bottom_right, color, thickness)

            # Draw filled white rectangle behind the text
            text_org = (top_left[0], top_left[1])
            text_box_top_left = text_org[0], text_org[1]
            text_box_bottom_right = text_org[0] + 50, text_org[1] + 20
            cv2.rectangle(frame, text_box_top_left, text_box_bottom_right, (255, 255, 255), cv2.FILLED)

            # Draw ID
            font = cv2.FONT_HERSHEY_SIMPLEX
            org = (top_left[0] + 5, top_left[1] + 15)
            font_scale = 0.6
            if class_id == 0:
 
                color = get_color(int(indiv_id))  # Color dependent on ID
            thickness = 2
            if class_id == 0:
                cv2.putText(frame, indivs[int(indiv_id)], org, font, font_scale, color, thickness)

    # Write the frame to the output video
    out.write(frame)

# Release the video capture and video writer
cap.release()
out.release()

print("Video processing completed.")


Video processing completed.
